<a href="https://colab.research.google.com/github/sura24378/Assignment-4/blob/main/Copy_of_final_project_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Collection

Part 2 of the project

Load Data

In [31]:
import pandas as pd
import numpy as np

# Load raw datasets
stocks = pd.read_csv('historical_stocks[1].csv')
prices = pd.read_csv('historical_stock_prices[1].csv')

# Convert date column
prices['date'] = pd.to_datetime(prices['date'])
prices.set_index('date', inplace=True)


 Advanced Data Cleaning

In [18]:
import os
print(os.listdir())  # Shows all files in the current directory


['.config', 'historical_stocks[1].csv', 'historical_stock_prices[1].csv', 'sample_data']


In [20]:
import pandas as pd

# Load stock prices dataset (make sure the file exists in your working directory)
prices = pd.read_csv('historical_stock_prices[1].csv')

# Show first few rows to verify it's loaded correctly
print(prices.head())

# Convert 'date' column to datetime and set as index
prices['date'] = pd.to_datetime(prices['date'])
prices.set_index('date', inplace=True)



  ticker   open  close  adj_close    low   high     volume        date
0    AHH  11.50  11.58   8.493155  11.25  11.68  4633900.0  2013-05-08
1    AHH  11.66  11.55   8.471151  11.50  11.66   275800.0  2013-05-09
2    AHH  11.55  11.60   8.507822  11.50  11.60   277100.0  2013-05-10
3    AHH  11.63  11.65   8.544494  11.55  11.65   147400.0  2013-05-13
4    AHH  11.60  11.53   8.456484  11.50  11.60   184100.0  2013-05-14


Detect & Resolve Outliers

In [21]:
# Interpolate missing numeric values
prices.interpolate(method='linear', inplace=True)

# Fill remaining missing values
prices.fillna(method='ffill', inplace=True)
prices.fillna(method='bfill', inplace=True)


/tmp/ipython-input-21-2496547360.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  prices.interpolate(method='linear', inplace=True)
/tmp/ipython-input-21-2496547360.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  prices.fillna(method='ffill', inplace=True)
/tmp/ipython-input-21-2496547360.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  prices.fillna(method='bfill', inplace=True)


In [22]:
from scipy.stats import zscore

# Define columns for outlier detection
outlier_cols = ['open', 'high', 'low', 'close', 'volume']

# Calculate Z-scores
z_scores = prices[outlier_cols].apply(zscore)

# Define outliers as Z > 4 (very extreme values)
outliers = (np.abs(z_scores) > 4)

# Cap outliers at 1st and 99th percentiles
for col in outlier_cols:
    lower, upper = prices[col].quantile([0.01, 0.99])
    prices[col] = np.clip(prices[col], lower, upper)


Error Detection & Correction

In [23]:
# Remove invalid price entries (e.g., negative prices or volume)
for col in ['open', 'high', 'low', 'close', 'volume']:
    prices = prices[prices[col] >= 0]

# Ensure logical consistency: low <= close <= high
prices = prices[(prices['low'] <= prices['close']) & (prices['close'] <= prices['high'])]


Data Transformation

In [24]:
# Rolling features
prices['rolling_close_7'] = prices['close'].rolling(window=7).mean()
prices['volatility_7'] = prices['close'].rolling(window=7).std()

# Technical indicator: price momentum (7-day difference)
prices['momentum_7'] = prices['close'].diff(7)


Standardization

In [25]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_cols = ['open', 'high', 'low', 'close', 'volume', 'rolling_close_7', 'volatility_7', 'momentum_7']
prices[scaled_cols] = scaler.fit_transform(prices[scaled_cols])


Encoding Categorical Variables

In [26]:
# Merge sector information
data = prices.reset_index().merge(stocks, on='ticker', how='left')

# One-hot encode 'sector'
data = pd.get_dummies(data, columns=['sector'], drop_first=True)


Integration and Modeling Preparation

In [27]:
# Drop rows with any missing engineered features
data.dropna(inplace=True)

# Re-index
data.set_index('date', inplace=True)


 Train/Validation/Test Split

In [28]:
from sklearn.model_selection import train_test_split

# Example: predict close price (as label)
X = data.drop(['close'], axis=1)
y = data['close']

# Split into train (70%), val (15%), test (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, shuffle=False)



 Save Cleaned Data

In [29]:
X_train.to_csv('X_train.csv')
X_val.to_csv('X_val.csv')
X_test.to_csv('X_test.csv')
y_train.to_csv('y_train.csv')
y_val.to_csv('y_val.csv')
y_test.to_csv('y_test.csv')


Data Cleaning & Transformation Summary
Missing Values: Imputed using linear interpolation and forward/backward filling to preserve time-series continuity.

Outliers: Identified using Z-scores; capped to the 1st and 99th percentiles to mitigate extreme influence without full removal.

Error Corrections: Removed rows with invalid price logic (e.g., negative values or inconsistent open/high/low/close structure).

Feature Engineering: Introduced rolling averages, volatility (standard deviation), and price momentum features.

Standardization: Numerical features standardized using z-score normalization.

Categorical Encoding: Applied one-hot encoding to sector data for model compatibility.

Splitting: Data divided into training (70%), validation (15%), and test (15%) sets chronologically.

Output: Cleaned and split datasets saved as CSV files for downstream modeling.

In [30]:
import pandas as pd
import numpy as np

# Load raw datasets
stocks = pd.read_csv('historical_stocks[1].csv')
prices = pd.read_csv('historical_stock_prices[1].csv')

# Convert date column
prices['date'] = pd.to_datetime(prices['date'])
prices.set_index('date', inplace=True)